In [20]:
import datetime as dt

from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *

from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *

from war_functions.pecota_tables import *
from war_functions.preseason_war_projections import *
from war_functions.preseason_win_percentage import *

In [2]:
# Rosters
rosters = retrieve_all_active_rosters()

In [3]:
# BOY cluster luck hitting

hitting_tables = retrieve_historical_hitting_tables([2021, 2019, 2018, 2017])
model = calculate_and_save_hitting_linear_regression(hitting_tables)
prev_year_hitting_table = retrieve_historical_hitting_tables(2021)
prev_year_hitting_adjustment = calculate_predicted_cluster_luck_run_adjustment_hitting(model, prev_year_hitting_table)

In [4]:
# BOY cluster luck pitching

pitching_tables = retrieve_historical_pitching_tables([2021, 2019, 2018, 2017])
model = calculate_and_save_pitching_linear_regression(pitching_tables)
prev_year_pitching_table = retrieve_historical_pitching_tables(2021)
prev_year_pitching_adjustment = calculate_predicted_cluster_luck_run_adjustment_pitching(model, prev_year_pitching_table)

In [5]:
# Merging cluster luck tables

final_cluster_luck = merge_cluster_luck_tables(prev_year_hitting_adjustment, prev_year_pitching_adjustment, file_name = 'data/final_cluster_luck.csv')

In [15]:
# Loading PECOTA data

pt = load_combined_pecota_table()

In [16]:
# Previous year war table

prev_year_war = retrieve_previous_year_war_table(2021)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST driver version for 100.0.4896
Get LATEST driver version for 100.0.4896
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64.zip
Driver has been saved in cache [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/100.0.4896.60]


,Team,WAR
0,Diamondbacks,22.3
1,Braves,45.4
2,Orioles,13.9
3,Red Sox,41.8
4,Cubs,28.2
5,Reds,41.8
6,Indians,27.5
7,Rockies,33.2
8,White Sox,36.1
9,Tigers,17.4


In [18]:
preseason_projections = calculate_preaseason_war_projections(rosters, pt)
final_war_preseason = calculate_final_preseason_war_change(preseason_projections, prev_year_war, current_year = 2022, file_name = 'data/overall_war_predictions_preseason.csv')

In [22]:
final_preseason_win_percentage = calculate_preseason_win_percentage(final_cluster_luck, final_war_preseason, current_year = 2022, file_name = 'data/preseason_projections.csv')